In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

def Getcmkey(url,uri):
    #https://www.cmoney.tw/finance/f00043.aspx?s=3413
    # 第一次查詢，為了取得cmkey 每隔一段時間會revoke
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    html = requests.get(url,headers=headers).text
    sp = BeautifulSoup(html,'html.parser')
    all_links = sp.find_all('a')

    for link in all_links:
        if link.get('href') == uri:
            cmkey=link.get('cmkey')
    
    cmkey=cmkey.replace('+','%2B')
    print('cmkey = '+cmkey)
    return cmkey

def GetFinancialRatios(stock):
    domain='https://www.cmoney.tw'
    uri = '/finance/f00043.aspx?s='+str(stock)
    url = domain + uri
    
    cmkey = Getcmkey(url,uri)
    # 加上取得的key ，即可查詢
    urladv = 'https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetFinancialRatios&stockId='+str(stock)+'&selectedType=4&cmkey='+str(cmkey)    
    ref = url
    
    # 偽瀏覽器，cmoney必需加上Referer header 才能抓取資料
    headersadv = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36','Referer': ref}
    
    # 下載該stock的財務比率表，並用pandas轉換成 dataframe
    r = requests.get(urladv, headers=headersadv)
    r.encoding = 'big5'
    print(r.url)
    dfs = pd.read_json(StringIO(r.text), encoding='big-5')

    return dfs

In [43]:
GetFinancialRatios(2884).loc[:,['DateRange','EPS','ROE']]

cmkey = SoNJ%2BxHYWX5ZWara9Ue8wg==
https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetFinancialRatios&stockId=2884&selectedType=4&cmkey=SoNJ%2BxHYWX5ZWara9Ue8wg==


,DateRange,EPS,ROE
0,201901,0.48,3.19
1,201804,0.35,2.35
2,201803,0.38,2.65
3,201802,0.41,2.91
4,201801,0.44,3.15
5,201704,0.29,2.16
6,201703,0.38,2.82
7,201702,0.36,2.75


In [33]:
dfn = GetFinancialRatios(2330).loc[:,['DateRange','EPS','ROE']]
EPSyear = dfn[(dfn['DateRange']>=201801) & (dfn['DateRange']<=201804)]
dfn

cmkey = SoNJ%2BxHYWX5ZWara9Ue8wg==
https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetFinancialRatios&stockId=2330&selectedType=4&cmkey=SoNJ%2BxHYWX5ZWara9Ue8wg==


,DateRange,EPS,ROE
0,201902,2.57,4.05
1,201901,2.37,3.59
2,201804,3.85,6.15
3,201803,3.44,5.81
4,201802,2.79,4.67
5,201801,3.46,5.74
6,201704,3.83,6.72
7,201703,3.47,6.49


In [38]:
def GetDividendPolicy(stock):
   
    domain='https://www.cmoney.tw'
    uri = '/finance/f00027.aspx?s='+str(stock)
    url = domain + uri
    
    cmkey = Getcmkey(url,uri)
    urladv = 'https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetDividendPolicy&stockId='+str(stock)+'&cmkey='+cmkey
    ref = url
    
    # 偽瀏覽器，cmoney必需加上Referer header 才能抓取資料
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36','Referer': ref}
    
    # 下載該stock的股利政策，並用pandas轉換成 dataframe
    r = requests.get(urladv, headers=headers)
    r.encoding = 'big5'
    print(r.url)
    dfs = pd.read_json(StringIO(r.text), encoding='big-5')
    #print(dfs)
    return dfs

In [39]:
GetDividendPolicy(2887)


cmkey = tMbttQidzOhnay%2Bwpm/RvA==
https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetDividendPolicy&stockId=2887&cmkey=tMbttQidzOhnay%2Bwpm/RvA==


,CashDividend,Date,ReserveAllotment,SurplusAllotment,TotalDividend,TotalStockDividend
0,0.51,2018,0,0.21,0.72,0.21
1,0.54,2017,0,0.44,0.99,0.44
2,0.53,2016,0,0.43,0.95,0.43
3,0.48,2015,0,0.72,1.21,0.72
4,0.10,2014,0,0.00,0.10,0.00
5,0.43,2013,0,0.99,1.42,0.99
6,0.22,2012,0,0.89,1.11,0.89
7,0.22,2011,0,0.90,1.12,0.90
8,0.23,2010,0,0.70,0.93,0.70
9,0.00,2009,0,1.10,1.10,1.10


In [59]:
def evaluate(stock,year):
    df1=GetDividendPolicy(stock)
    df2=GetFinancialRatios(stock)
    #print(df1)
    Dividend = df1[df1['Date']==year]['TotalDividend']
    #EPSyear =  df2.loc[2:5,'EPS'].sum()
    EPSyear = df2[(df2['DateRange']>=201801) & (df2['DateRange']<=201804)]['EPS'].sum()
    
    test=Dividend/EPSyear
    #近四季EPS
    EPS =  df2.loc[0:3,'EPS'].sum()
    ROE =  df2.loc[0:3,'ROE'].sum()
    #盈餘成長率
    result = ROE*(1-Dividend/EPSyear)
    print('股利總和 = %.2f'%Dividend)
    print('盈餘配發率 = %.3f'%test)
    print('2018年度EPS = %.2f'%EPSyear)
    print('近四季ROE = %.2f'%ROE)
    print('近四季EPS = %.2f'%EPS)
    print('盈餘成長率 = %.2f'%result)

    buyprice = result*0.75*EPS
    sellprice = result*EPS

    print('買進參考價 = %.2f'%buyprice)
    print('賣出參考價 = %.2f'%sellprice)

In [60]:
import twstock
evaluate(3481,2018)
lastprice = twstock.Stock('3481').price[-1:]
print(lastprice)
print(type(lastprice))

cmkey = tMbttQidzOhnay%2Bwpm/RvA==
https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetDividendPolicy&stockId=3481&cmkey=tMbttQidzOhnay%2Bwpm/RvA==
cmkey = SoNJ%2BxHYWX5ZWara9Ue8wg==
https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetFinancialRatios&stockId=3481&selectedType=4&cmkey=SoNJ%2BxHYWX5ZWara9Ue8wg==
股利總和 = 0.06
盈餘配發率 = 0.261
2018年度EPS = 0.23
近四季ROE = -2.18
近四季EPS = -0.55
盈餘成長率 = -1.61
買進參考價 = 0.66
賣出參考價 = 0.89
[7.04]
<class 'list'>


In [56]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

stock=2317
domain='https://www.cmoney.tw'
uri = '/finance/f00043.aspx?s='+str(stock)
url = domain + uri
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

html = requests.get(url,headers=headers).text
sp = BeautifulSoup(html,'html.parser')
all_links = sp.find_all('a')

for link in all_links:
    if link.get('href') == uri:
        cmkey=link.get('cmkey')

print(cmkey)

urladv = 'https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetFinancialRatios&stockId='+str(stock)+'&selectedType=4&cmkey='+str(cmkey)

ref = url

    # 偽瀏覽器，cmoney必需加上Referer header 才能抓取資料
headersadv = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36','Referer': ref}
    
    # 下載該stock的財務比率表，並用pandas轉換成 dataframe
    # 將url 中的 + 取代為 %2b 才不會出錯
r = requests.get(urladv.replace('+','%2b'), headers=headersadv)
r.encoding = 'big5'
print(r.url)

try:
    dfs = pd.read_json(StringIO(r.text), encoding='big-5')
except:
    print('web site error!!!')




SoNJ+xHYWX5ZWara9Ue8wg==
https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetFinancialRatios&stockId=2317&selectedType=4&cmkey=SoNJ%2bxHYWX5ZWara9Ue8wg==
